# Interpreting DICE2016R2 Results

In [1]:
using DICE
using FileIO
using DataFrames

In [2]:
base = load("R2BaseResults.jld2");

In [3]:
quint_ψ₂ = [0.00061, 0.00141, 0.00227, 0.00242, 0.00464]; #Coefficients for the damage equation kkdam
quint_gₐ₀ = [-0.00241, 0.04307, 0.076, 0.09843, 0.16508]; #Coefficients for the PROD equation kkprod
quint_t2xco2 = [2.00767, 2.51586, 3.1, 3.38802, 4.49126]; #Coefficients for the climate equation kkets
quint_gσ₀ = [-0.02002, -0.01689, -0.0152, -0.01334, -0.01055]; #Coefficients for emissions intensity kksig
quint_mueq = [233.59, 292.95, 360, 394.35, 519.33]; #Coefficients for the carbon cycle kkcarb

mappedphi = Array{Float64,1}();
mappedt2 = Array{Float64,1}();
mappedga = Array{Float64,1}();
mappedmueq = Array{Float64,1}();
mappedgs = Array{Float64,1}();
for phi in 1 #quint_ψ₂
    for t2 in 1 #quint_t2xco2
        for ga in 1 #quint_gₐ₀
            for mu in 1 #quint_mueq
                for gs in 1:5 #quint_gσ₀
                    push!(mappedphi, quint_ψ₂[phi]);
                    push!(mappedt2, quint_t2xco2[t2]);
                    push!(mappedga, quint_gₐ₀[ga]);
                    push!(mappedmueq, quint_mueq[mu]);
                    push!(mappedgs, quint_gσ₀[gs]);
                end
            end
        end
    end
end


In [4]:
#1 -> 2015, 2 -> 2020, 8 -> 2050, 18 -> 2100, 38 -> 2200
results = DataFrame(damcoef = mappedphi
                  , etscoef = mappedt2
                  , prodcoef = mappedga # This is prod2100 in the xls, but I'm not sure where that is mapped from
                  , carbcoef = mappedmueq
                  , sigcoef = mappedgs
                  , scc2015 = map(r -> r.scc[1],  values(base))
                  , scc2020 = map(r -> r.scc[2],  values(base))
                  , temp2050 = map(r -> r.Tₐₜ[8],  values(base))
                  , temp2100 = map(r -> r.Tₐₜ[18],  values(base))
                  , temp2200 = map(r -> r.Tₐₜ[38],  values(base))
                  , carbonconc2050 = map(r -> r.Mₐₜ[8],  values(base))
                  , carbonconc2100 = map(r -> r.Mₐₜ[18],  values(base))
                  , carbonconc2200 = map(r -> r.Mₐₜ[38],  values(base))
                  , output2050 = map(r -> r.Y[8],  values(base))
                  , output2100 = map(r -> r.Y[18],  values(base))
                  , emis2050 = map(r -> r.Eind[8],  values(base))
                  , emis2100 = map(r -> r.Eind[18],  values(base))
                  , emis2200 = map(r -> r.Eind[38],  values(base))
                  , damfrac2050 = map(r -> r.Ω[8],  values(base))
                  , damfrac2100 = map(r -> r.Ω[18],  values(base))
                  , damfrac2200 = map(r -> r.Ω[38],  values(base))
                  , sigma2050 = map(r -> r.Σ[8],  values(base))
                  , sigma2100 = map(r -> r.Σ[18],  values(base))
                  , sigma2200 = map(r -> r.Σ[38],  values(base))
                  , r2015 = map(r -> r.RI[1],  values(base))
                  , r2100 = map(r -> r.RI[18],  values(base))
                  , Objective = map(r -> r.UTILITY,  values(base))
                  , cca2100 = map(r -> r.CCA[18],  values(base))
                  , cca2200 = map(r -> r.CCA[38],  values(base))
                  , cca2300 = map(r -> r.CCA[58],  values(base))
                  , ppm2050 = map(r -> r.Mₐₜppm[8],  values(base))
                  , ppm2100 = map(r -> r.Mₐₜppm[18],  values(base))
                  , ppm2200 = map(r -> r.Mₐₜppm[38],  values(base))
    );

In [5]:
results

,damcoef,etscoef,prodcoef,carbcoef,sigcoef,scc2015,scc2020,temp2050,temp2100,temp2200,carbonconc2050,carbonconc2100,carbonconc2200,output2050,output2100,emis2050,emis2100,emis2200,damfrac2050,damfrac2100,damfrac2200,sigma2050,sigma2100,sigma2200,r2015,r2100,Objective,cca2100,cca2200,cca2300,ppm2050,ppm2100,ppm2200
1,0.00061,2.00767,-0.00241,233.59,-0.02002,7.97594,9.0706,1.84004,2.59499,2.75676,1201.89,1363.43,1424.73,158.297,175.228,26.4298,10.6668,1.73957,0.0020653,0.00410771,0.00463584,0.166617,0.0606219,0.00988857,0.0369838,0.0144855,-958.662,970.492,1109.38,1133.13,564.266,640.11,668.886
2,0.00061,2.00767,-0.00241,233.59,-0.01689,7.99654,9.08888,1.86787,2.74725,3.11355,1229.59,1469.9,1635.16,158.257,175.076,33.2693,18.4024,5.394,0.00212826,0.0046039,0.00591344,0.209773,0.104621,0.0306807,0.0369819,0.0144644,-962.356,1109.7,1405.86,1495.39,577.272,690.093,767.681
3,0.00061,2.00767,-0.00241,233.59,-0.0152,7.99206,9.0805,1.88039,2.82175,3.3127,1242.41,1526.18,1768.91,158.238,174.996,36.6255,23.1086,8.66361,0.00215688,0.00485698,0.00669412,0.230956,0.131402,0.0492973,0.0369816,0.0144523,-964.479,1182.73,1592.53,1749.63,583.292,716.516,830.474
4,0.00061,2.00767,-0.00241,233.59,-0.01334,7.99433,9.08813,1.85983,2.70134,2.99891,1221.47,1436.6,1563.64,158.268,175.124,31.2044,15.8101,3.93487,0.00210997,0.00445131,0.00548601,0.196742,0.0898734,0.0223767,0.0369823,0.0144713,-961.158,1066.32,1305.52,1367.23,573.459,674.461,734.102
5,0.00061,2.00767,-0.00241,233.59,-0.01055,7.98962,9.08412,1.85272,2.66206,2.9057,1214.37,1408.94,1508.26,158.279,175.163,29.4394,13.7726,2.95512,0.00209387,0.00432279,0.00515028,0.185605,0.0782842,0.0168024,0.0369827,0.0144768,-960.193,1030.18,1227.55,1271.63,570.125,661.473,708.105


In [6]:
colwise(mean, results), colwise(std, results)

([0.00061, 2.00767, -0.00241, 233.59, -0.0152, 7.9897, 9.08245, 1.86017, 2.70548, 3.01752  …  0.0258091, 0.0369824, 0.0144701, -961.37, 1071.88, 1328.17, 1403.4, 573.683, 676.53, 741.849], [0.0, 0.0, 0.0, 0.0, 0.00357668, 0.00811128, 0.00742412, 0.0152413, 0.0856949, 0.210474  …  0.0151805, 8.66432e-7, 1.25883e-5, 2.20244, 80.2605, 183.323, 234.549, 7.18693, 28.8977, 61.3266])